In [1]:
import numpy as np
import pandas as pd
import joblib
import re

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


from keras.models import load_model

# Load data and model

In [2]:
df_sentence = pd.read_pickle('../dataset/SS/clean/sentence_data.pkl')

sentence_corpus = {}
for d in df_sentence:
    sentence_corpus.update(d)
    
corpus = pd.read_pickle('../dataset/SS/clean/SPECTER_embeddings.pkl')
kmeans_5 = joblib.load('../model/kmeans_5_specter.model')
pca = joblib.load('../model/pca.model')

# Workflow
### paper -> embedding -> pca -> kmeans -> cluster_label -> choose model -> recommend citations -> get sentences -> process sentences -> generate related work section

In [5]:
# Get embedding
embedding = corpus.loc['42490916', 'SPECTER_embeddings'].reshape(1, -1)

# PCA transform
vec = pca.transform(embedding)

# kmeans cluster
label = kmeans_5.predict(vec)

# choose a model and citation list
filename = '../model/model_' + str(label[0]) + '.h5'
citename = '../dataset/SS/clean/citation_' + str(label[0]) + '.pkl'

model = load_model(filename)
df_citation = pd.read_pickle(citename)
citations = np.array(df_citation['citation_id'])

prediction = model.predict(embedding)[0]
recommendation = np.argsort(prediction)[::-1][:20] # recommend 20 possible citations

# retrieve sentences and remove cite spans.
rw_sentence = []
for idx in citations[recommendation]:
    s = sentence_corpus.get(idx)
    if s is not None:
        s = re.sub(r' \[.*\]', '', s)
        rw_sentence.append(s)
        
generated_rw = ' '.join(rw_sentence)

In [6]:
print('Original citations: \n', corpus.loc['42490916', 'rw_citations'], '\n')
print('Recommend citations: \n', citations[recommendation])

Original citations: 
 ['11864530', '3961724', '16510219', '6485959', '7973738', '425268'] 

Recommend citations: 
 ['206593071' '425268' '7973738' '3961724' '938105' '14909631' '14997888'
 '11864530' '6485959' '14124239' '3198903' '9088600' '206595765'
 '12203312' '15172651' '206590483' '10070153' '14991802' '8964627'
 '3332134']


In [7]:
print('Original related work section: \n')
print(corpus.loc['42490916', 'relatedwork_text'])

Original related work section: 

Person Re-identification A lot of methods have been proposed for person re-identification and can be divided into two categories: direct methods and learning based methods. Direct methods focus on feature descriptors or geometric structure to describe pedestrians with no requirements of learning. Farenzena et al. [10] take advantage of the symmetry and asymmetry of human structure and propose Symmetry-Driven Accumulation of Local Features (SDALF) including weighted-HSV histogram, Maximally Stable Color Regions (MSCR), and Recurrent Highly Structured Patches (RHSP). Ma et al. [11] achieve superior performance by combining BiCov descriptor with SDALF to handle illumination change. Cheng et al. [12] utilize Pictorial Structures (PS) to pay attention to body parts and takes advantage of part-to-part correspondence. On the other hand, learning methods always use pre-labled training sets to learn proper features or metric functions [13] , [14] , [24] , [25] ,

In [8]:
print('Generated related work section: \n')
print(generated_rw)

Generated related work section: 

The first group uses the siamese model with image pairs as inputs. Most existing identity alignment methods focus on supervised learning of person identity-discriminative information. In ) and 3) recurrent highly structured patches (RHSP). Early person re-identification methods focus on manually designing discriminative features . Methods based on Siamese models alone are not a viable solution to our problem, since they need image crops of the object and can not fully utilize re-identification annotations due to their pairwise labelling training setup. Concerning multiple-shot approaches, in the spatiotemporal graph was generated for a ten consecutive frames for grouping spatiotemporally similar regions. The first group uses the siamese model with image pairs as inputs. Early person re-identification methods focus on manually designing discriminative features . Before deep neural networks spring up, there are a great amount of research efforts for desi